## Monopoly Game Agent (v3)

The following notebook creates agents to automatically play Monopoly. You can
specify the behaviour of these agents in two ways:
- How the agents decide whether to buy a property
- How the agents decide whether to build a house/hotel on a property

At present, more extensive decision-making features (like trading or mortgaging) are not included. However, these could be added in future versions.

### Project Setup (Do not edit)

Install dependencies

In [11]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Import dependencies

In [12]:
# monopoly simulator
from simulator.monosim.player import Player
from simulator.monosim.board import get_board, get_roads, get_properties, get_community_chest_cards, get_bank

# LangChain
from langchain_openai import ChatOpenAI
from langchain import OpenAI, LLMChain, PromptTemplate

# other imports
from pydantic import BaseModel, Field
from typing import List

# dotenv for loading API keys
import os
from dotenv import load_dotenv

In [13]:
# Load API keys from .env file
load_dotenv()

# Define the API keys
openai_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# Fast and slow minds
def fast_mind(prompt: str, model="gpt-4o-mini") -> dict:
    """Fast mind for quick decisions."""
    model_instance = ChatOpenAI(model=model, api_key=openai_key)
    response = model_instance.predict(prompt)
    uncertainty = float(response.metadata.get("uncertainty", 0.0))
    decision = response.text.strip()
    return {"decision": decision, "uncertainty": uncertainty}

def slow_mind(prompt: str, model="gpt-4o") -> dict:
    """Slow mind for complex decisions."""
    model_instance = ChatOpenAI(model=model, api_key=openai_key)
    response = model_instance.predict(prompt)
    decision = response.text.strip()
    return {"decision": decision}

### Buying Properties (Edit here)

This function can be modified to get the agent to behave differently when deciding whether to buy a property. The placeholders are just examples of what you can do. Note: you **don't need to use all the information available when making a decision**.

In [14]:
def custom_buy(self, dict_property_info : dict) -> str | None:
    """
    Edit this function to determine whether the player should buy a property. 

    Return:
    YOUR FUNCTION MUST RETURN "buy" (type str) to buy the property 
    or None (type None) to not buy the property.
    
    WARNING: I've included exhaustive documentation on all the data you can 
    use to make this decision. DON'T use all of it; it's mostly irrelevant. 

    Parameters:
    self: useful for accessing the following methods
    - self.get_state(): returns player data in a dict with the keys
        - cash: the amount of cash the player has
        - owned_roads: a list of owned roads
        - owned_stations: a list of owned stations
        - owned_utilities: a list of owned utilities
        - owned_colors: a dict which maps colors to True if the player owns all 
            properties of that color. Ex: owned_colors['blue'] = True/False
        - owned_houses_hotels: a dict which maps property names to the number of 
            houses/hotels built there. Ex: owned_houses_hotels['Park Place'] = 3
        - mortgageable_amount: total mortgageable value of player's properties
        WARNING: PROBABLY NOT USEFUL DATA BELOW
        - morgaged_roads: a list of mortgaged roads
        - mortgaged_stations: a list of mortgaged stations
        - mortgaged_utilities: a list of mortgaged utilities
        - name: name of player
        - number: player number
        - position: player's position on the board
        - dice_value: the sum of the dice rolled
        - jail_count: the number of turns the player has been in jail
        - exit_jail: whether the player has exited jail
        - free_visit: whether the player is visiting jail for free
        - has_lost: whether the player has lost the game
    - self._dict_properties[property_name]: returns property info for an 
    arbitrary property with a dict like dict_property_info below

    dict_property_info: Has a dict with info about the property to buy/not buy. 
        The dict has the following keys:
    * name: the name of the property
    * belongs_to: the player who owns the property
    * price: the price of the property
    * rent: the rent of the property
    - color: the color of the property
    - rent_with_color_set: rent when all properties of same color are owned
    * type: the type of property (one of 'road', 'station', 'utility')
    - houses_cost: the cost of building a house
    - hotels_cost: the cost of building a hotel
    - rent_with_X_houses_0_hotels: rent when X = {1, 2, 3, 4} houses are built
    - rent_with_4_houses_1_hotels: rent when 4 houses and 1 hotel are built
    WARNING: PROBABLY NOT USEFUL DATA BELOW
    * mortgage_value: the amount of money earned when mortgaging the property
    * unmortgage_value: the amount of money cost to unmortgage the property
    * is_mortgaged: whether the property is mortgaged
    * board_num: the position of the property on the board
    P.S. (* means for all property types, - means for road properties only)
    """

    # currently, no trading. So don't buy property if someone else owns it
    if dict_property_info['belongs_to'] != None:
        return None

    # If the property isn't owned, I get the LLM to decide whether to buy it
    # Steps: 1. making a prompt template, 2. injecting data, 3. setting up 
    # output parsing, and 4. running an LLM.

    ##################################################
    # Step 1: Making a prompt template
    ##################################################

    # I'm trying to keep this prompt template general to reuse it for other games
    template = """
    You are a strategic decision-maker playing a game of {game}.

    Rules of the game:
    {rules} 
    
    Your goal:
    Maximise your chances of winning {game} with forward-thinking reasoning.

    Your task:
    Analyse whether to buy {property_name} given the data that follows.

    Property Information:
    - Cost: {price}
    - Base rent: {rent}
    
    Provide an uncertainty score reflecting your confidence in the decision. The score should be accurate and align with your analysis.
    
    Reason whether you should buy {property_name}. Then output yes/no as your final answer.
    """

    # Only add color data when available (for roads)
    if (dict_property_info['type'] == 'road'):
        template += "\n- Color: {color}\n- Number of {color} properties you own: {n_color_properties}\n\nYou currently have {cash} in cash and {n_roads} roads."
    elif (dict_property_info['type'] == 'station'):
        template += "\n\nYou currently have {cash} in cash and {n_stations} stations."
    elif (dict_property_info['type'] == 'utility'):
        template += "\n\nYou currently have {cash} in cash and {n_utilities} utilities."
    template += "\nReason whether you should buy {property_name}. Then output yes/no as your final answer."

    
    ##################################################
    # Step 2: Injecting data
    ##################################################

    inject = {
        'agent_name': self.get_state()['name'],
        'game': 'Monopoly',
        # Insert better rules later
        'rules': 'Typical Monopoly rules, but you cannot trade with other players or mortgage properties.',
        'property_name': dict_property_info['name'],
        'price': dict_property_info['price'],
        'rent': dict_property_info['rent'],
        'cash': self.get_state()['cash'],
        'n_roads': len(self.get_state()['owned_roads']),
        'n_stations': len(self.get_state()['owned_stations']),
        'n_utilities': len(self.get_state()['owned_utilities'])
    }

    # Only add color data when available (for roads)
    if (dict_property_info['type'] == 'road'):
        inject['color'] = dict_property_info['color']
        
        # Bugs here! self._dict_properties.get(property) fails for some
        # properties. Don't reuse this code!
        n_color_properties = 0
        for property in self.get_state()['owned_roads']:
            prop_info = self._dict_properties.get(property)
            if prop_info and prop_info['color'] == dict_property_info['color']:
                n_color_properties += 1
        inject['n_color_properties'] = n_color_properties

    prompt_template = PromptTemplate(input_variables=inject.keys(), template=template)
    prompt = prompt_template.format(**inject)

    
    ##################################################
    # Step 3: Setting up output parsing
    ##################################################

    class Output(BaseModel):
        reasoning: str = Field(description="Your reasoning for the decision")
        decision: str = Field(description="Whether to buy the property (yes/no)")
    
    ##################################################
    # Step 4: Running the LLM
    ##################################################

# Fast mind decision
    fast_response = fast_mind(prompt)

    # Check uncertainty
    if fast_response['uncertainty'] > 60.0:

        slow_response = slow_mind(prompt)
        decision = slow_response['decision']
    else:
        decision = fast_response['decision']

    return "buy" if decision == "yes" else None

### Backend Setup (Do not edit)

These are functions that setup a game and let you access the state of the game.

In [15]:
def modify_buy_or_bid(buy) -> str:
    return custom_buy

class CustomPlayer(Player):
    buy_or_bid = modify_buy_or_bid(Player.buy_or_bid)

In [16]:
def initialize_game() -> dict:
    """
    Initializes a game with two players and sets up the bank, board, roads, properties, 
    and community chest cards.
    
    Returns:
        dict: A dictionary containing the following:
            - "bank": Game's bank object.
            - "board": Main game board.
            - "roads": List of road objects.
            - "properties": List of property objects.
            - "community_chest_cards": Dictionary of community chest cards.
            - "players": List of two Player objects, with Player 1 first.
    """
    
    bank = get_bank()
    board = get_board()
    roads = get_roads()
    properties = get_properties()
    community_chest_cards = get_community_chest_cards()
    community_cards_deck = list(community_chest_cards.keys())

    # Note how we have one of our players vs. a default player that just buys 
    # whenever cash is available. We can change this later
    player1 = CustomPlayer('Alice', 1, bank, board, roads, properties, community_cards_deck)
    player2 = Player('Bob', 2, bank, board, roads, properties, community_cards_deck)
    
    player1.meet_other_players([player2])
    player2.meet_other_players([player1])
    
    return {
        "bank": bank,
        "board": board,
        "roads": roads,
        "properties": properties,
        "community_chest_cards": community_chest_cards,
        "players": [player1, player2]
    }

In [17]:
def get_current_state(players) -> dict:
    """
    Retrieves the current state of each player, including position, owned roads, 
    money, mortgaged properties, and other status details.

    Args:
        players (list[Player]): List of Player objects in the game.

    Returns:
        dict: A dictionary containing:
            - "players": A list of dictionaries, each with a player's state.
    """
    
    current_state = {
        "players": [{"state": player.get_state()} for player in players]
    }
    return current_state

### Run a Game (Be careful - it costs real money!)

In [18]:
game = initialize_game()

In [19]:
player1 = game["players"][0]
player2 = game["players"][1]
list_players = [player1, player2]

# WARNING: KEEP THIS UNDER 10 ROUNDS FOR NOW
# EACH ROUND COSTS US MONEY
STOP_AT_ROUND = 2

In [20]:
idx_count = 0
while not player1.has_lost() and not player2.has_lost() and idx_count < STOP_AT_ROUND:
    for player in list_players:
        # Uncomment this line to run the game if you're sure there are no bugs
        player.play() 
    idx_count += 1

reasoning='Buying Whitechapel Road for 60 will reduce my cash to 1440, which is still a healthy amount. The base rent of 4 is low, but owning this property can help me acquire the entire brown set if I manage to buy the other brown property in the future. The brown properties are relatively low-cost investments, and owning them typically provides a good return on investment when developed. Given these factors, purchasing Whitechapel Road is a strategic move for future development and potential rent income.' decision='yes'
reasoning='Buying Euston Road for 100 will increase my property portfolio, which is beneficial as it allows me to collect rent from opponents landing on it. The base rent of 6 is relatively low, but it is a light blue property, and owning multiple light blue properties can increase the rent significantly. Given that I have enough cash (1440) to buy it without affecting my liquidity, it is a strategic move to acquire it and potentially build upon it later. Thus, purcha